In [ ]:
## imports
import os
os.chdir(os.pardir)

import matplotlib.pyplot as plt
import jax
import jax.numpy as np
import pandas as pd
from scipy.interpolate import CloughTocher2DInterpolator

from pycbem.field import efield, hfield
from pycbem.utils.dataloader import (load_antenna_el_properties,
                                     load_head_coords)
from pycbem.utils.viz import fig_config

In [ ]:
jax.config.update("jax_enable_x64", True)

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
# frequency
f = 10e9

# effective radiated skin area
target_area = (0.02, 0.02)  # 2 x 2 cm2
A = target_area[0] * target_area[1]
d = - 15 / 1000  # distance from the antenna

# human head
head_coords = load_head_coords()
target_area_coords = head_coords[
    (head_coords['y'] < 0) &
    (head_coords['x'] > 0) & (head_coords['x'] < target_area[0]) &
    (head_coords['z'] > 0) & (head_coords['z'] < target_area[1])]
target_area_coords.reset_index(drop=True, inplace=True)

# antenna electric properties (Poljak 2005)
antenna_data = load_antenna_el_properties(f)
Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j

# antenna position -- coordinates
xs = antenna_data.x.to_numpy() 
xs = xs + (target_area[0] - xs[-1]) / 2.
ys = np.zeros_like(xs) + target_area_coords['y'].min() + d
zs = np.zeros_like(xs)
zs = zs + (target_area[1] - zs[-1]) / 2.

In [ ]:
# E field
E = target_area_coords.apply(
    lambda row: efield(row['x'], row['y'], row['z'], xs, ys, zs, Is, f),
    axis=1, result_type='expand')
E.columns = ['Ex', 'Ey', 'Ez']
E_abs = E.apply(
    lambda row: np.sqrt(row['Ex'] ** 2 + row['Ey'] ** 2 + row['Ez'] ** 2),
    axis=1)
E.loc[:, 'E_abs'] = E_abs

# H field
H = target_area_coords.apply(
    lambda row: hfield(row['x'], row['y'], row['z'], xs, ys, zs, Is, f),
    axis=1, result_type='expand')
H.columns = ['Hx', 'Hy', 'Hz']
H_abs = H.apply(
    lambda row: np.sqrt(row['Hx'] ** 2 + row['Hy'] ** 2 + row['Hz'] ** 2),
    axis=1)
H.loc[:, 'H_abs'] = H_abs

# update dataframe
target_area_calc = pd.concat([target_area_coords, E, H], axis=1)

# power density
def _pd(row):
    return (row['Ey'] * row['Hz'].conjugate()
            - row['Ez'] * row['Hy'].conjugate(),
            row['Ex'] * row['Hz'].conjugate(),
            row['Ex'] * row['Hy'].conjugate())

S = target_area_calc.apply(_pd, axis=1, result_type='expand')
S.columns = ['Sx', 'Sy', 'Sz']
S_abs = S.apply(
    lambda row: np.sqrt(row['Sx'] ** 2 + row['Sy'] ** 2 + row['Sz'] ** 2),
    axis=1)
S.loc[:, 'S_abs'] = S_abs 

# update dataframe
target_area_calc = pd.concat([target_area_calc, S], axis=1)

In [ ]:
x = target_area_calc['x'].to_numpy().reshape(-1, 1)
y = target_area_calc['y'].to_numpy().reshape(-1, 1)
z = target_area_calc['z'].to_numpy().reshape(-1, 1)
xz_plane_inp = np.hstack((x, z))
i = np.abs(target_area_calc['S_abs'].to_numpy(dtype=np.complex128))
xz_interp_fn = CloughTocher2DInterpolator(xz_plane_inp, i)
xz_grid = np.meshgrid(np.linspace(x.min(), x.max(), 101),
                      np.linspace(z.min(), z.max(), 101))
i_xz = xz_interp_fn(*xz_grid)

In [ ]:
fig_config(latex=True, scaler=1.5)
fig = plt.figure()
ax = fig.add_subplot()
cs = ax.imshow(i_xz, origin='lower', extent=(x.min(), x.max(), z.min(), z.max()))
ax.plot(xs, zs, 'r-', linewidth=2, label='dipole antenna')
cbar = fig.colorbar(cs, ax=ax)
cbar.ax.set_ylabel(r'S [W/m$^2$]')
ax.set_xlabel('$x$ [m]')
ax.set_ylabel('$z$ [m]')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
fig_config(latex=True, ncols=2, scaler=1.5)
fig, ax = plt.subplots(1, 2, sharey=True, squeeze=True)
ax[0].scatter(head_coords['x'], head_coords['z'], color='k', alpha=0.03)
ax[0].contourf(*xz_grid, i_xz, levels=11)
ax[0].plot(xs, zs, 'r-', linewidth=2, label='1/2 dipole (xz-plane)')
ax[0].set_xlabel('$x$ [m]')
ax[0].set_ylabel('$z$ [m]')
ax[1].scatter(head_coords['y'], head_coords['z'], color='k', alpha=0.03)
im = ax[1].scatter(target_area_calc['y'], target_area_calc['z'],
                   c=np.abs(target_area_calc['S_abs'].to_numpy(dtype='complex128')))
ax[1].plot(ys[0], zs[0], 'ro', label='1/2 wave dipole (yz-plane)')
ax[1].set_xlabel('$y$ [m]')

fig.colorbar(im, ax=ax, label='S [W/m$^2$]')
fig.legend(bbox_to_anchor=(0.55, 1.11), ncol=1, frameon=True, edgecolor='k')

plt.show()

In [ ]:
# singular value decomposition
a = target_area_calc[['Sx', 'Sy', 'Sz']].to_numpy(dtype='complex128')
u, s, vh = np.linalg.svd(a, full_matrices=False)
s = np.diag(s)

# singular values
fig_config(latex=True, ncols=2, scaler=1.5)
fig = plt.figure()

ax1 = fig.add_subplot(1, 2, 1)
ax1.semilogy(np.diag(s), 'o-')
ax1.set_title('singular values')
ax1.set_xticks([0, 1, 2])

ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(np.cumsum(np.diag(s)) / np.sum(s), 'o-')
ax2.set_title('signular value total energy')
ax2.set_xticks([0, 1, 2])

plt.show()

In [ ]:
# truncation: 3d -> 2d
rank = 2
a_trunc = u[:, :rank] @ s[:rank, :rank] @ vh[:rank, :rank]